<a href="https://colab.research.google.com/github/hoangviet2/My-Chess-Engine/blob/main/ChessEngineModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install maia2


In [2]:
pip install chess einops gdown numpy pandas pyzstd Requests torch tqdm pyyaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 52.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.6/413.6 kB 30.5 MB/s eta 0:00:00
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=0207c60308d60a12657d333c89759d7fbe4651306a51891a7d5ba998fdbea549
  Stored in directory: /root/.cache/pip/wheels/83/1f/4e/8f4300f7dd554eb8de70ddfed96e94d3d030ace10c5b53d447
Successfully built chess


In [3]:
from maia2 import model, dataset, inference

In [ ]:
data = dataset.load_example_test_dataset()

Downloading...
From: https://drive.google.com/uc?id=1fSu4Yp8uYj7xocbHAbjBP6DthsgiJy9X
To: /content/maia2_data/example_test_dataset.csv
100%|██████████| 22.5M/22.5M [00:00<00:00, 69.2MB/s]


Example test dataset downloaded.


In [ ]:
maia2_model = model.from_pretrained(type="rapid", device="gpu")

Model for rapid games already downloaded.
Model for rapid games loaded to gpu.


In [ ]:
data, acc = inference.inference_batch(data, maia2_model, verbose=1, batch_size=1024, num_workers=4)
print(acc)

100%|██████████| 105/105 [01:18<00:00,  1.33it/s]


0.5311


In [4]:
from maia2 import train, utils

In [17]:
import requests
import re

def download_all_games(username: str, output_file: str = "all_games.pgn"):
    headers = {"User-Agent": "MyChessDownloader/1.0"}

    # Step 1: Get archives
    archives_url = f"https://api.chess.com/pub/player/{username}/games/archives"
    response = requests.get(archives_url, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Error fetching archives for {username} (HTTP {response.status_code})")

    archives = response.json().get("archives", [])

    if not archives:
        print("⚠️ No archives found for this user.")
        return

    # Step 2: Fetch PGNs
    with open(output_file, "w", encoding="utf-8") as outfile:
        for url in archives:
            pgn_url = f"{url}/pgn"
            print(f"Fetching {pgn_url} ...")

            pgn_response = requests.get(pgn_url, headers=headers)
            if pgn_response.status_code == 200:
                pgn_text = pgn_response.text
                pgn_text = re.sub(r'(\[%clk \d+:\d+:\d+)\.\d+\]', r'\1]', pgn_text)


            # 🔥 Add "Rapid" to Event line so it passes your filter
                games = pgn_text.strip().split("\n\n\n")  # Chess.com separates games with triple newlines
                for g in games:
                    if '[Variant "' in g:  # ❌ skip non-standard variants
                        continue
                    outfile.write(g.strip() + "\n\n\n")
                pgn_text = pgn_text.replace('[Event "Live Chess"]',
                                        '[Event "Live Chess Rated Rapid"]')
                #outfile.write(pgn_response.text + "\n\n")
            else:
                print(f"⚠️ Skipped {pgn_url} (HTTP {pgn_response.status_code})")

    print(f"✅ All games saved to {output_file}")


In [ ]:
download_all_games("sukhoi35","lichess_db_standard_rated_2025-06.pgn")

Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/05/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/06/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/07/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/08/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/09/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/10/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/11/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2016/12/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2017/01/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2017/02/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2017/03/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2017/04/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/games/2017/05/pgn ...
Fetching https://api.chess.com/pub/player/sukhoi35/

In [ ]:
import chess.pgn

def validate_pgn(file_path):
    with open(file_path, encoding="utf-8") as f:
        game_num = 1
        while True:
            try:
                game = chess.pgn.read_game(f)
                if game is None:
                    break
                game_num += 1
            except Exception as e:
                print(f"❌ Error parsing game {game_num}: {e}")
                break


In [ ]:
validate_pgn("/content/all_games.pgn")

In [5]:
pip install zstandard

In [ ]:
import zstandard as zstd

def compress_pgn(input_file: str, output_file: str):
    """
    Compress a PGN file into .pgn.zst format using Zstandard.
    """
    with open(input_file, "rb") as f_in, open(output_file, "wb") as f_out:
        cctx = zstd.ZstdCompressor(level=3)  # level 1-22 (higher = more compression, slower)
        f_out.write(cctx.compress(f_in.read()))

    print(f"✅ Compressed {input_file} -> {output_file}")

In [ ]:
compress_pgn("lichess_db_standard_rated_2025-06.pgn","lichess_db_standard_rated_2015-06.pgn.zst")

✅ Compressed lichess_db_standard_rated_2013-01.pgn -> lichess_db_standard_rated_2013-01.pgn.zst


In [ ]:
from multiprocessing import Process, Queue, cpu_count
cfg = utils.parse_args(cfg_file_path="/content/config.yaml")
pgn_chunks = utils.read_or_create_chunks("/content/lichess_db_standard_rated_2013-01.pgn", cfg)
print("Number of chunks:", len(pgn_chunks))
pgn_chunks_sublists = []
num_processes = cpu_count() - cfg.num_cpu_left
for i in range(0, len(pgn_chunks), num_processes):
                pgn_chunks_sublists.append(pgn_chunks[i:i + num_processes])
print(pgn_chunks_sublists[0])

Loading cached chunks from /content/lichess_db_standard_rated_2013-01_chunks.pkl
Number of chunks: 12134
[(0, 6844), (6844, 14776)]


In [9]:
cfg = utils.parse_args(cfg_file_path="/content/drive/MyDrive/ChessEngine/config.yaml")
train.run(cfg)

Configurations:
	data_root: /content/drive/MyDrive/ChessEngine
	seed: 42
	num_workers: 16
	verbose: 1
	max_epochs: 3
	max_ply: 300
	clock_threshold: 30
	chunk_size: 250
	start_year: 2025
	start_month: 1
	end_year: 2025
	end_month: 6
	from_checkpoint: False
	checkpoint_epoch: 0
	checkpoint_year: 2025
	checkpoint_month: 8
	num_cpu_left: 0
	queue_length: 1
	lr: 0.0001
	wd: 1e-05
	batch_size: 8192
	first_n_moves: 10
	last_n_moves: 10
	dim_cnn: 256
	dim_vit: 1024
	num_blocks_cnn: 5
	num_blocks_vit: 2
	input_channels: 18
	vit_length: 8
	elo_dim: 128
	side_info: True
	side_info_coefficient: 1.0
	value: True
	value_coefficient: 1.0
	max_games_per_elo_range: 50000
MAIA2Model(
  (chess_cnn): ChessResNet(
    (conv1): Conv2d(18, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layers): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [6]:
def from_pretrained(type, device, save_root = "/saves/0.0001_8192_1e-05"):

    if os.path.exists(save_root) == False:
        os.makedirs(save_root)

    if os.path.exists(output_path):
        print(f"Model for {type} games already downloaded.")
    else:
        print(f"Downloading model for {type} games.")
        gdown.download(url, output_path, quiet=False)

    cfg_url = "https://drive.google.com/uc?id=1GQTskYMVMubNwZH2Bi6AmevI15CS6gk0"
    cfg_path = os.path.join(save_root, "config.yaml")
    if not os.path.exists(cfg_path):
        gdown.download(cfg_url, cfg_path, quiet=False)

    cfg = parse_args(cfg_path)

    all_moves = get_all_possible_moves()
    elo_dict = create_elo_dict()

    model = MAIA2Model(len(all_moves), elo_dict, cfg)
    model = nn.DataParallel(model)

    checkpoint = torch.load(output_path, map_location='cpu')
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.module

    if device == "gpu":
        model = model.cuda()

    print(f"Model for {type} games loaded to {device}.")

    return model

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import torch
from torch import nn
from maia2.main import MAIA2Model
from maia2.utils import get_all_possible_moves, create_elo_dict, parse_args

def load_my_model(checkpoint_path, config_path, device="cpu"):
    # Parse config file
    cfg = parse_args(config_path)

    # Create model
    all_moves = get_all_possible_moves()
    elo_dict = create_elo_dict()
    model = MAIA2Model(len(all_moves), elo_dict, cfg)
    model = nn.DataParallel(model)  # same as in repo

    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    model = model.module  # unwrap DataParallel

    # Send to GPU if needed
    if device == "gpu":
        model = model.cuda()

    print(f"Custom Maia-2 model loaded from {checkpoint_path} to {device}")
    return model

In [ ]:
my_checkpoint = "/content/drive/MyDrive/ChessEngine/epoch_1_2013-01.pgn.pt"
my_config    = "/content/drive/MyDrive/ChessEngine/config.yaml"
my_model = load_my_model(my_checkpoint, my_config, device="cpu")

Custom Maia-2 model loaded from /content/drive/MyDrive/ChessEngine/epoch_1_2013-01.pgn.pt to cpu


In [ ]:
from maia2 import dataset

# Example test dataset included in repo
data = dataset.load_example_test_dataset()

Downloading...
From: https://drive.google.com/uc?id=1fSu4Yp8uYj7xocbHAbjBP6DthsgiJy9X
To: /content/maia2_data/example_test_dataset.csv
100%|██████████| 22.5M/22.5M [00:00<00:00, 204MB/s]


Example test dataset downloaded.


In [ ]:
from maia2 import inference

data, acc = inference.inference_batch(
    data,
    my_model,
    verbose=1,       # progress bar
    batch_size=1024, # adjust depending on GPU memory
    num_workers=4
)

print("Accuracy:", acc)

100%|██████████| 105/105 [39:55<00:00, 22.81s/it]


Accuracy: 0.1061


Fix the origin

In [9]:
import pdb
import chess
import pickle
import os
import random
import numpy as np
import torch
import time
import requests
import tqdm
import pyzstd
import re
import yaml


class Config:

    def __init__(self, config_dict):
        for key, value in config_dict.items():
            setattr(self, key, value)


def parse_args(cfg_file_path):

    with open(cfg_file_path, 'r') as f:
        cfg_dict = yaml.safe_load(f)

    cfg = Config(cfg_dict)

    return cfg


def seed_everything(seed: int):

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def delete_file(filename):

    if os.path.exists(filename):
        os.remove(filename)
        print(f"Data {filename} has been deleted.")
    else:
        print(f"The file '{filename}' does not exist.")


def readable_num(num):

    if num >= 1e9:  # if parameters are in the billions
        return f'{num / 1e9:.2f}B'
    elif num >= 1e6:  # if parameters are in the millions
        return f'{num / 1e6:.2f}M'
    elif num >= 1e3:  # if parameters are in the thousands
        return f'{num / 1e3:.2f}K'
    else:
        return str(num)


def readable_time(elapsed_time):

    hours, rem = divmod(elapsed_time, 3600)
    minutes, seconds = divmod(rem, 60)

    if hours > 0:
        return f"{int(hours)}h {int(minutes)}m {seconds:.2f}s"
    elif minutes > 0:
        return f"{int(minutes)}m {seconds:.2f}s"
    else:
        return f"{seconds:.2f}s"


def count_parameters(model):

    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

    return readable_num(total_params)


def create_elo_dict():

    inteval = 100
    start = 1100
    end = 2000

    range_dict = {f"<{start}": 0}
    range_index = 1

    for lower_bound in range(start, end - 1, inteval):
        upper_bound = lower_bound + inteval
        range_dict[f"{lower_bound}-{upper_bound - 1}"] = range_index
        range_index += 1

    range_dict[f">={end}"] = range_index

    # print(range_dict, flush=True)

    return range_dict


def map_to_category(elo, elo_dict):

    inteval = 100
    start = 1100
    end = 2000

    if elo < start:
        return elo_dict[f"<{start}"]
    elif elo >= end:
        return elo_dict[f">={end}"]
    else:
        for lower_bound in range(start, end - 1, inteval):
            upper_bound = lower_bound + inteval
            if lower_bound <= elo < upper_bound:
                return elo_dict[f"{lower_bound}-{upper_bound - 1}"]


def get_side_info(board, move_uci, all_moves_dict):
    move = chess.Move.from_uci(move_uci)

    moving_piece = board.piece_at(move.from_square)
    captured_piece = board.piece_at(move.to_square)

    from_square_encoded = torch.zeros(64)
    from_square_encoded[move.from_square] = 1

    to_square_encoded = torch.zeros(64)
    to_square_encoded[move.to_square] = 1

    if move_uci == 'e1g1':
        rook_move = chess.Move.from_uci('h1f1')
        from_square_encoded[rook_move.from_square] = 1
        to_square_encoded[rook_move.to_square] = 1

    if move_uci == 'e1c1':
        rook_move = chess.Move.from_uci('a1d1')
        from_square_encoded[rook_move.from_square] = 1
        to_square_encoded[rook_move.to_square] = 1

    board.push(move)
    is_check = board.is_check()
    board.pop()

    # Order: Pawn, Knight, Bishop, Rook, Queen, King
    side_info = torch.zeros(6 + 6 + 1)
    side_info[moving_piece.piece_type - 1] = 1
    if move_uci in ['e1g1', 'e1c1']:
        side_info[3] = 1
    if captured_piece:
        side_info[6 + captured_piece.piece_type - 1] = 1
    if is_check:
        side_info[-1] = 1

    legal_moves = torch.zeros(len(all_moves_dict))
    legal_moves_idx = torch.tensor([all_moves_dict[move.uci()] for move in board.legal_moves])
    legal_moves[legal_moves_idx] = 1

    side_info = torch.cat([side_info, from_square_encoded, to_square_encoded, legal_moves], dim=0)

    return legal_moves, side_info


def extract_clock_time(comment):

    match = re.search(r'\[%clk (\d+):(\d+):(\d+)\]', comment)
    if match:
        hours, minutes, seconds = map(int, match.groups())
        return hours * 3600 + minutes * 60 + seconds
    return None


def read_or_create_chunks(pgn_path, cfg):

    cache_file = pgn_path.replace('.pgn', '_chunks.pkl')

    if os.path.exists(cache_file):
        print(f"Loading cached chunks from {cache_file}")
        with open(cache_file, 'rb') as f:
            pgn_chunks = pickle.load(f)
    else:
        print(f"Cache not found. Creating chunks for {pgn_path}")
        start_time = time.time()
        pgn_chunks = get_chunks(pgn_path, cfg.chunk_size)
        print(f'Chunking took {readable_time(time.time() - start_time)}', flush=True)

        with open(cache_file, 'wb') as f:
            pickle.dump(pgn_chunks, f)

    return pgn_chunks


def board_to_tensor(board):

    piece_types = [chess.PAWN, chess.KNIGHT, chess.BISHOP, chess.ROOK, chess.QUEEN, chess.KING]
    num_piece_channels = 12  # 6 piece types * 2 colors
    additional_channels = 6  # 1 for player's turn, 4 for castling rights, 1 for en passant
    tensor = torch.zeros((num_piece_channels + additional_channels, 8, 8), dtype=torch.float32)

    # Precompute indices for each piece type
    piece_indices = {piece: i for i, piece in enumerate(piece_types)}

    # Fill tensor for each piece type
    for piece_type in piece_types:
        for color in [True, False]:  # True is White, False is Black
            piece_map = board.pieces(piece_type, color)
            index = piece_indices[piece_type] + (0 if color else 6)
            for square in piece_map:
                row, col = divmod(square, 8)
                tensor[index, row, col] = 1.0

    # Player's turn channel (White = 1, Black = 0)
    turn_channel = num_piece_channels
    if board.turn == chess.WHITE:
        tensor[turn_channel, :, :] = 1.0

    # Castling rights channels
    castling_rights = [board.has_kingside_castling_rights(chess.WHITE),
                       board.has_queenside_castling_rights(chess.WHITE),
                       board.has_kingside_castling_rights(chess.BLACK),
                       board.has_queenside_castling_rights(chess.BLACK)]
    for i, has_right in enumerate(castling_rights):
        if has_right:
            tensor[num_piece_channels + 1 + i, :, :] = 1.0

    # En passant target channel
    ep_channel = num_piece_channels + 5
    if board.ep_square is not None:
        row, col = divmod(board.ep_square, 8)
        tensor[ep_channel, row, col] = 1.0

    return tensor

def generate_pawn_promotions():
    # Define the promotion rows for both colors and the promotion pieces
    # promotion_rows = {'white': '7', 'black': '2'}
    promotion_rows = {'white': '7'}
    promotion_pieces = ['q', 'r', 'b', 'n']
    promotions = []

    # Iterate over each color
    for color, row in promotion_rows.items():
        # Target rows for promotion (8 for white, 1 for black)
        target_row = '8' if color == 'white' else '1'

        # Each file from 'a' to 'h'
        for file in 'abcdefgh':
            # Direct move to promotion
            for piece in promotion_pieces:
                promotions.append(f'{file}{row}{file}{target_row}{piece}')

            # Capturing moves to the left and right (if not on the edges of the board)
            if file != 'a':
                left_file = chr(ord(file) - 1)  # File to the left
                for piece in promotion_pieces:
                    promotions.append(f'{file}{row}{left_file}{target_row}{piece}')

            if file != 'h':
                right_file = chr(ord(file) + 1)  # File to the right
                for piece in promotion_pieces:
                    promotions.append(f'{file}{row}{right_file}{target_row}{piece}')

    return promotions


def mirror_square(square):

    file = square[0]
    rank = str(9 - int(square[1]))

    return file + rank


def mirror_move(move_uci):
    # Check if the move is a promotion (length of UCI string will be more than 4)
    is_promotion = len(move_uci) > 4

    # Extract the start and end squares, and the promotion piece if applicable
    start_square = move_uci[:2]
    end_square = move_uci[2:4]
    promotion_piece = move_uci[4:] if is_promotion else ""

    # Mirror the start and end squares
    mirrored_start = mirror_square(start_square)
    mirrored_end = mirror_square(end_square)

    # Return the mirrored move, including the promotion piece if applicable
    return mirrored_start + mirrored_end + promotion_piece


def get_chunks(pgn_path, chunk_size):

    chunks = []
    with open(pgn_path, 'r', encoding='utf-8') as pgn_file:
        while True:
            start_pos = pgn_file.tell()
            game_count = 0
            while game_count < chunk_size:
                line = pgn_file.readline()
                if not line:
                    break
                if line[-4:] == "1-0\n" or line[-4:] == "0-1\n":
                    game_count += 1
                if line[-8:] == "1/2-1/2\n":
                    game_count += 1
                if line[-2:] == "*\n":
                    game_count += 1
            line = pgn_file.readline()
            if line not in ["\n", ""]:
                raise ValueError
            end_pos = pgn_file.tell()
            chunks.append((start_pos, end_pos))
            if not line:
                break

    return chunks


def decompress_zst(file_path, decompressed_path):
    """ Decompress a .zst file using pyzstd """
    with open(file_path, 'rb') as compressed_file, open(decompressed_path, 'wb') as decompressed_file:
        pyzstd.decompress_stream(compressed_file, decompressed_file)


def get_all_possible_moves():

    all_moves = []

    for rank in range(8):
        for file in range(8):
            square = chess.square(file, rank)

            board = chess.Board(None)
            board.set_piece_at(square, chess.Piece(chess.QUEEN, chess.WHITE))
            legal_moves = list(board.legal_moves)
            all_moves.extend(legal_moves)

            board = chess.Board(None)
            board.set_piece_at(square, chess.Piece(chess.KNIGHT, chess.WHITE))
            legal_moves = list(board.legal_moves)
            all_moves.extend(legal_moves)

    all_moves = [all_moves[i].uci() for i in range(len(all_moves))]

    pawn_promotions = generate_pawn_promotions()

    return all_moves + pawn_promotions


def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [10]:
import torch
import torch.nn as nn
import pdb
import argparse
import os
from multiprocessing import Process, Queue, cpu_count
import time
def run(cfg):

    print('Configurations:', flush=True)
    for arg in vars(cfg):
        print(f'\t{arg}: {getattr(cfg, arg)}', flush=True)
    seed_everything(cfg.seed)
    num_processes = cpu_count() - cfg.num_cpu_left

    save_root = f'/content/drive/MyDrive/ChessEngine/'
    if not os.path.exists(save_root):
        os.makedirs(save_root)

    all_moves = get_all_possible_moves()
    all_moves_dict = {move: i for i, move in enumerate(all_moves)}
    elo_dict = create_elo_dict()

    model = MAIA2Model(len(all_moves), elo_dict, cfg)

    print(model, flush=True)
    model = model.cuda()
    model = nn.DataParallel(model)
    criterion_maia = nn.CrossEntropyLoss()
    criterion_side_info = nn.BCEWithLogitsLoss()
    criterion_value = nn.MSELoss()

    optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.wd)
    N_params = count_parameters(model)
    print(f'Trainable Parameters: {N_params}', flush=True)

    accumulated_samples = 0
    accumulated_games = 0

    if cfg.from_checkpoint:
        formatted_month = f"{cfg.checkpoint_month:02d}"
        checkpoint = torch.load(save_root + f'epoch_{cfg.checkpoint_epoch}_{cfg.checkpoint_year}-{formatted_month}.pgn.pt')
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        accumulated_samples = checkpoint['accumulated_samples']
        accumulated_games = checkpoint['accumulated_games']

    for epoch in range(cfg.max_epochs):

        print(f'Epoch {epoch + 1}', flush=True)
        pgn_paths = read_monthly_data_path(cfg)

        num_file = 0
        for pgn_path in pgn_paths:

            start_time = time.time()
            decompress_zst(pgn_path + '.zst', pgn_path)
            print(f'Decompressing {pgn_path} took {readable_time(time.time() - start_time)}', flush=True)

            pgn_chunks = read_or_create_chunks(pgn_path, cfg)
            print(f'Training {pgn_path} with {len(pgn_chunks)} chunks.', flush=True)

            queue = Queue(maxsize=cfg.queue_length)

            pgn_chunks_sublists = []
            for i in range(0, len(pgn_chunks), num_processes):
                pgn_chunks_sublists.append(pgn_chunks[i:i + num_processes])

            pgn_chunks_sublist = pgn_chunks_sublists[0]
            # For debugging only
            # process_chunks(cfg, pgn_path, pgn_chunks_sublist, elo_dict)
            worker = Process(target=preprocess_thread, args=(queue, cfg, pgn_path, pgn_chunks_sublist, elo_dict))
            worker.start()

            num_chunk = 0
            offset = 0
            while True:
                if not queue.empty():
                    if offset + 1 < len(pgn_chunks_sublists):
                        pgn_chunks_sublist = pgn_chunks_sublists[offset + 1]
                        worker = Process(target=preprocess_thread, args=(queue, cfg, pgn_path, pgn_chunks_sublist, elo_dict))
                        worker.start()
                        offset += 1
                    data, game_count, chunk_count = queue.get()
                    loss, loss_maia, loss_side_info, loss_value = train_chunks(cfg, data, model, optimizer, all_moves_dict, criterion_maia, criterion_side_info, criterion_value)
                    num_chunk += chunk_count
                    accumulated_samples += len(data)
                    accumulated_games += game_count
                    print(f'[{num_chunk}/{len(pgn_chunks)}]', flush=True)
                    print(f'[# Positions]: {readable_num(accumulated_samples)}', flush=True)
                    print(f'[# Games]: {readable_num(accumulated_games)}', flush=True)
                    print(f'[# Loss]: {loss} | [# Loss MAIA]: {loss_maia} | [# Loss Side Info]: {loss_side_info} | [# Loss Value]: {loss_value}', flush=True)
                    if num_chunk == len(pgn_chunks):
                        break

            num_file += 1
            print(f'### ({num_file} / {len(pgn_paths)}) Took {readable_time(time.time() - start_time)} to train {pgn_path} with {len(pgn_chunks)} chunks.', flush=True)
            os.remove(pgn_path)

            torch.save({'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'accumulated_samples': accumulated_samples,
                        'accumulated_games': accumulated_games}, f'{save_root}epoch_{epoch + 1}_{pgn_path[-11:]}.pt')

In [15]:
import chess.pgn
import chess
import pdb
from multiprocessing import Pool, cpu_count, Queue, Process
import torch
import tqdm
import torch.nn as nn
import torch.nn.functional as F
from tqdm.contrib.concurrent import process_map
import os
import pandas as pd
import time
from einops import rearrange


def process_chunks(cfg, pgn_path, pgn_chunks, elo_dict):

    # process_per_chunk((pgn_chunks[0][0], pgn_chunks[0][1], pgn_path, elo_dict, cfg))

    if cfg.verbose:
        results = process_map(process_per_chunk, [(start, end, pgn_path, elo_dict, cfg) for start, end in pgn_chunks], max_workers=len(pgn_chunks), chunksize=1)
    else:
        with Pool(processes=len(pgn_chunks)) as pool:
            results = pool.map(process_per_chunk, [(start, end, pgn_path, elo_dict, cfg) for start, end in pgn_chunks])

    ret = []
    count = 0
    list_of_dicts = []
    for result, game_count, frequency in results:
        ret.extend(result)
        count += game_count
        list_of_dicts.append(frequency)

    total_counts = {}

    for d in list_of_dicts:
        for key, value in d.items():
            total_counts[key] = total_counts.get(key, 0) + value

    print(total_counts, flush=True)

    return ret, count, len(pgn_chunks)


def process_per_game(game, white_elo, black_elo, white_win, cfg):

    ret = []

    board = game.board()
    moves = list(game.mainline_moves())

    for i, node in enumerate(game.mainline()):

        move = moves[i]

        if i >= cfg.first_n_moves:

            comment = node.comment
            clock_info = extract_clock_time(comment)

            if i % 2 == 0:
                board_input = board.fen()
                move_input = move.uci()
                elo_self = white_elo
                elo_oppo = black_elo
                active_win = white_win

            else:
                board_input = board.mirror().fen()
                move_input = mirror_move(move.uci())
                elo_self = black_elo
                elo_oppo = white_elo
                active_win = - white_win

            #if clock_info > cfg.clock_threshold:
            ret.append((board_input, move_input, elo_self, elo_oppo, active_win))

        board.push(move)
        if i == cfg.max_ply:
            break

    return ret


def game_filter(game):

    white_elo = game.headers.get("WhiteElo", "?")
    black_elo = game.headers.get("BlackElo", "?")
    time_control = game.headers.get("TimeControl", "?")
    result = game.headers.get("Result", "?")
    event = game.headers.get("Event", "?")

    if white_elo == "?" or black_elo == "?" or time_control == "?" or result == "?" or event == "?":
        return

    if 'Rated' not in event:
        return

    if 'Rapid' not in event:
        return

    #for _, node in enumerate(game.mainline()):
        #if 'clk' not in node.comment:
            #return

    white_elo = int(white_elo)
    black_elo = int(black_elo)

    if result == '1-0':
        white_win = 1
    elif result == '0-1':
        white_win = -1
    elif result == '1/2-1/2':
        white_win = 0
    else:
        return

    return game, white_elo, black_elo, white_win


def process_per_chunk(args):

    start_pos, end_pos, pgn_path, elo_dict, cfg = args

    ret = []
    game_count = 0

    frequency = {}

    with open(pgn_path, 'r', encoding='utf-8') as pgn_file:

        pgn_file.seek(start_pos)

        while pgn_file.tell() < end_pos:

            game = chess.pgn.read_game(pgn_file)

            if game is None:
                break

            filtered_game = game_filter(game)
            if filtered_game:
                game, white_elo, black_elo, white_win = filtered_game
                white_elo = map_to_category(white_elo, elo_dict)
                black_elo = map_to_category(black_elo, elo_dict)

                if white_elo < black_elo:
                    range_1, range_2 = black_elo, white_elo
                else:
                    range_1, range_2 = white_elo, black_elo

                freq = frequency.get((range_1, range_2), 0)
                if freq >= cfg.max_games_per_elo_range:
                    continue

                ret_per_game = process_per_game(game, white_elo, black_elo, white_win, cfg)
                ret.extend(ret_per_game)
                if len(ret_per_game):

                    if (range_1, range_2) in frequency:
                        frequency[(range_1, range_2)] += 1
                    else:
                        frequency[(range_1, range_2)] = 1

                    game_count += 1

    return ret, game_count, frequency


class MAIA1Dataset(torch.utils.data.Dataset):

    def __init__(self, data, all_moves_dict, elo_dict, cfg):

        self.all_moves_dict = all_moves_dict
        self.cfg = cfg
        self.data = data.values.tolist()
        self.elo_dict = elo_dict

    def __len__(self):

        return len(self.data)

    def __getitem__(self, idx):

        fen, move, elo_self, elo_oppo, white_active = self.data[idx]

        if white_active:
            board = chess.Board(fen)
        else:
            board = chess.Board(fen).mirror()
            move = mirror_move(move)

        board_input = board_to_tensor(board)
        move_input = self.all_moves_dict[move]

        elo_self = map_to_category(elo_self, self.elo_dict)
        elo_oppo = map_to_category(elo_oppo, self.elo_dict)

        legal_moves, side_info = get_side_info(board, move, self.all_moves_dict)

        return board_input, move_input, elo_self, elo_oppo, legal_moves, side_info


class MAIA2Dataset(torch.utils.data.Dataset):


    def __init__(self, data, all_moves_dict, cfg):

        self.all_moves_dict = all_moves_dict
        self.data = data
        self.cfg = cfg

    def __len__(self):

        return len(self.data)

    def __getitem__(self, idx):

        board_input, move_uci, elo_self, elo_oppo, active_win = self.data[idx]

        board = chess.Board(board_input)
        board_input = board_to_tensor(board)

        legal_moves, side_info = get_side_info(board, move_uci, self.all_moves_dict)

        move_input = self.all_moves_dict[move_uci]

        return board_input, move_input, elo_self, elo_oppo, legal_moves, side_info, active_win


class BasicBlock(torch.nn.Module):

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        mid_planes = planes

        self.conv1 = torch.nn.Conv2d(in_planes, mid_planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(mid_planes)
        self.conv2 = torch.nn.Conv2d(mid_planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = torch.nn.BatchNorm2d(planes)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):

        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out)
        out = self.dropout(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out += x
        out = F.relu(out)

        return out


class ChessResNet(torch.nn.Module):

    def __init__(self, block, cfg):
        super(ChessResNet, self).__init__()

        self.conv1 = torch.nn.Conv2d(cfg.input_channels, cfg.dim_cnn, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(cfg.dim_cnn)
        self.layers = self._make_layer(block, cfg.dim_cnn, cfg.num_blocks_cnn)
        self.conv_last = torch.nn.Conv2d(cfg.dim_cnn, cfg.vit_length, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn_last = torch.nn.BatchNorm2d(cfg.vit_length)

    def _make_layer(self, block, planes, num_blocks, stride=1):

        layers = []
        for _ in range(num_blocks):
            layers.append(block(planes, planes, stride))

        return torch.nn.Sequential(*layers)

    def forward(self, x):

        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layers(out)
        out = self.conv_last(out)
        out = self.bn_last(out)

        return out


class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim, dropout = 0.):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)


class EloAwareAttention(nn.Module):
    def __init__(self, dim, heads=8, dim_head=64, dropout=0., elo_dim=64):
        super().__init__()
        inner_dim = dim_head * heads
        project_out = not (heads == 1 and dim_head == dim)

        self.heads = heads
        self.scale = dim_head ** -0.5

        self.norm = nn.LayerNorm(dim)

        self.attend = nn.Softmax(dim=-1)
        self.dropout = nn.Dropout(dropout)

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias=False)

        self.elo_query = nn.Linear(elo_dim, inner_dim, bias=False)

        self.to_out = nn.Sequential(
            nn.Linear(inner_dim, dim),
            nn.Dropout(dropout)
        ) if project_out else nn.Identity()

    def forward(self, x, elo_emb):
        x = self.norm(x)

        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h=self.heads), qkv)

        elo_effect = self.elo_query(elo_emb).view(x.size(0), self.heads, 1, -1)
        q = q + elo_effect

        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale

        attn = self.attend(dots)
        attn = self.dropout(attn)

        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)


class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim, dropout = 0., elo_dim=64):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([])
        self.elo_layers = nn.ModuleList([])
        for _ in range(depth):
            self.elo_layers.append(nn.ModuleList([
                EloAwareAttention(dim, heads = heads, dim_head = dim_head, dropout = dropout, elo_dim = elo_dim),
                FeedForward(dim, mlp_dim, dropout = dropout)
            ]))

    def forward(self, x, elo_emb):
        for attn, ff in self.elo_layers:
            x = attn(x, elo_emb) + x
            x = ff(x) + x

        return self.norm(x)


class MAIA2Model(torch.nn.Module):

    def __init__(self, output_dim, elo_dict, cfg):
        super(MAIA2Model, self).__init__()

        self.cfg = cfg
        self.chess_cnn = ChessResNet(BasicBlock, cfg)

        heads = 16
        dim_head = 64
        self.to_patch_embedding = nn.Sequential(
            nn.Linear(8 * 8, cfg.dim_vit),
            nn.LayerNorm(cfg.dim_vit),
        )
        self.transformer = Transformer(cfg.dim_vit, cfg.num_blocks_vit, heads, dim_head, mlp_dim=cfg.dim_vit, dropout = 0.1, elo_dim = cfg.elo_dim * 2)
        self.pos_embedding = nn.Parameter(torch.randn(1, cfg.vit_length, cfg.dim_vit))

        self.fc_1 = nn.Linear(cfg.dim_vit, output_dim)
        # self.fc_1_1 = nn.Linear(cfg.dim_vit, cfg.dim_vit)
        self.fc_2 = nn.Linear(cfg.dim_vit, output_dim + 6 + 6 + 1 + 64 + 64)
        # self.fc_2_1 = nn.Linear(cfg.dim_vit, cfg.dim_vit)
        self.fc_3 = nn.Linear(128, 1)
        self.fc_3_1 = nn.Linear(cfg.dim_vit, 128)

        self.elo_embedding = torch.nn.Embedding(len(elo_dict), cfg.elo_dim)

        self.dropout = nn.Dropout(p=0.1)
        self.last_ln = nn.LayerNorm(cfg.dim_vit)


    def forward(self, boards, elos_self, elos_oppo):

        batch_size = boards.size(0)
        boards = boards.view(batch_size, self.cfg.input_channels, 8, 8)
        embs = self.chess_cnn(boards)
        embs = embs.view(batch_size, embs.size(1), 8 * 8)
        x = self.to_patch_embedding(embs)
        x += self.pos_embedding
        x = self.dropout(x)

        elos_emb_self = self.elo_embedding(elos_self)
        elos_emb_oppo = self.elo_embedding(elos_oppo)
        elos_emb = torch.cat((elos_emb_self, elos_emb_oppo), dim=1)
        x = self.transformer(x, elos_emb).mean(dim=1)

        x = self.last_ln(x)

        logits_maia = self.fc_1(x)
        logits_side_info = self.fc_2(x)
        logits_value = self.fc_3(self.dropout(torch.relu(self.fc_3_1(x)))).squeeze(dim=-1)

        return logits_maia, logits_side_info, logits_value


def read_monthly_data_path(cfg):

    print('Training Data:', flush=True)
    pgn_paths = []

    for year in range(cfg.start_year, cfg.end_year + 1):
        start_month = cfg.start_month if year == cfg.start_year else 1
        end_month = cfg.end_month if year == cfg.end_year else 12

        for month in range(start_month, end_month + 1):
            formatted_month = f"{month:02d}"
            pgn_path = cfg.data_root + f"/lichess_db_standard_rated_{year}-{formatted_month}.pgn"
            # skip 2019-12
            if year == 2019 and month == 12:
                continue
            print(pgn_path, flush=True)
            pgn_paths.append(pgn_path)

    return pgn_paths


def evaluate(model, dataloader):

    counter = 0
    correct_move = 0

    model.eval()
    with torch.no_grad():

        for boards, labels, elos_self, elos_oppo, legal_moves, side_info in dataloader:

            boards = boards.cuda()
            labels = labels.cuda()
            elos_self = elos_self.cuda()
            elos_oppo = elos_oppo.cuda()
            legal_moves = legal_moves.cuda()

            logits_maia, logits_side_info, logits_value = model(boards, elos_self, elos_oppo)
            logits_maia_legal = logits_maia * legal_moves
            preds = logits_maia_legal.argmax(dim=-1)
            correct_move += (preds == labels).sum().item()

            counter += len(labels)

    return correct_move, counter


def evaluate_MAIA1_data(model, all_moves_dict, elo_dict, cfg, tiny=False):

    elo_list = range(1000, 2600, 100)

    for i in elo_list:
        start = i
        end = i + 100
        file_path = f"../data/test/KDDTest_{start}-{end}.csv"
        data = pd.read_csv(file_path)
        data = data[data.type == 'Rapid'][['board', 'move', 'active_elo', 'opponent_elo', 'white_active']]
        dataset = MAIA1Dataset(data, all_moves_dict, elo_dict, cfg)
        dataloader = torch.utils.data.DataLoader(dataset,
                                                batch_size=cfg.batch_size,
                                                shuffle=False,
                                                drop_last=False,
                                                num_workers=cfg.num_workers)
        if cfg.verbose:
            dataloader = tqdm.tqdm(dataloader)
        print(f'Testing Elo Range {start}-{end} with MAIA 1 data:', flush=True)
        correct_move, counter = evaluate(model, dataloader)
        print(f'Accuracy Move Prediction: {round(correct_move / counter, 4)}', flush=True)
        if tiny:
            break


def train_chunks(cfg, data, model, optimizer, all_moves_dict, criterion_maia, criterion_side_info, criterion_value):

    dataset_train = MAIA2Dataset(data, all_moves_dict, cfg)
    dataloader_train = torch.utils.data.DataLoader(dataset_train,
                                                    batch_size=cfg.batch_size,
                                                    shuffle=True,
                                                    drop_last=False,
                                                    num_workers=cfg.num_workers)
    if cfg.verbose:
        dataloader_train = tqdm.tqdm(dataloader_train)

    avg_loss = 0
    avg_loss_maia = 0
    avg_loss_side_info = 0
    avg_loss_value = 0
    step = 0
    for boards, labels, elos_self, elos_oppo, legal_moves, side_info, wdl in dataloader_train:

        model.train()
        boards = boards.cuda()
        labels = labels.cuda()
        elos_self = elos_self.cuda()
        elos_oppo = elos_oppo.cuda()
        side_info = side_info.cuda()
        wdl = wdl.float().cuda()

        logits_maia, logits_side_info, logits_value = model(boards, elos_self, elos_oppo)

        loss = 0
        loss_maia = criterion_maia(logits_maia, labels)
        loss += loss_maia

        if cfg.side_info:

            loss_side_info = criterion_side_info(logits_side_info, side_info) * cfg.side_info_coefficient
            loss += loss_side_info

        if cfg.value:
            loss_value = criterion_value(logits_value, wdl) * cfg.value_coefficient
            loss += loss_value

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()
        avg_loss_maia += loss_maia.item()
        if cfg.side_info:
            avg_loss_side_info += loss_side_info.item()
        if cfg.value:
            avg_loss_value += loss_value.item()
        step += 1

    return round(avg_loss / step, 3), round(avg_loss_maia / step, 3), round(avg_loss_side_info / step, 3), round(avg_loss_value / step, 3)


def preprocess_thread(queue, cfg, pgn_path, pgn_chunks_sublist, elo_dict):

    data, game_count, chunk_count = process_chunks(cfg, pgn_path, pgn_chunks_sublist, elo_dict)
    queue.put([data, game_count, chunk_count])
    del data


def worker_wrapper(semaphore, *args, **kwargs):
    with semaphore:
        preprocess_thread(*args, **kwargs)


In [16]:
cfg = parse_args(cfg_file_path="/content/drive/MyDrive/ChessEngine/config.yaml")
run(cfg)

Configurations:
	data_root: /content/drive/MyDrive/ChessEngine
	seed: 42
	num_workers: 16
	verbose: 1
	max_epochs: 3
	max_ply: 300
	clock_threshold: 30
	chunk_size: 250
	start_year: 2025
	start_month: 1
	end_year: 2025
	end_month: 6
	from_checkpoint: False
	checkpoint_epoch: 0
	checkpoint_year: 2025
	checkpoint_month: 8
	num_cpu_left: 0
	queue_length: 1
	lr: 0.0001
	wd: 1e-05
	batch_size: 8192
	first_n_moves: 10
	last_n_moves: 10
	dim_cnn: 256
	dim_vit: 1024
	num_blocks_cnn: 5
	num_blocks_vit: 2
	input_channels: 18
	vit_length: 8
	elo_dim: 128
	side_info: True
	side_info_coefficient: 1.0
	value: True
	value_coefficient: 1.0
	max_games_per_elo_range: 50000
MAIA2Model(
  (chess_cnn): ChessResNet(
    (conv1): Conv2d(18, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layers): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1

  0%|          | 0/4 [00:00<?, ?it/s]

{(10, 10): 254}


100%|██████████| 4/4 [00:09<00:00,  2.49s/it]

[12/1241]
[# Positions]: 26.98K
[# Games]: 272
[# Loss]: 9.168 | [# Loss MAIA]: 7.608 | [# Loss Side Info]: 0.697 | [# Loss Value]: 0.863


...

[1116/1117]
[# Positions]: 10.18M
[# Games]: 100.29K
[# Loss]: 3.487 | [# Loss MAIA]: 2.885 | [# Loss Side Info]: 0.025 | [# Loss Value]: 0.576



100%|██████████| 1/1 [00:00<00:00,  1.10it/s]

[1117/1117]
[# Positions]: 10.18M
[# Games]: 100.30K
[# Loss]: 3.612 | [# Loss MAIA]: 3.05 | [# Loss Side Info]: 0.026 | [# Loss Value]: 0.536
### (5 / 6) Took 12m 58.59s to train /content/drive/MyDrive/ChessEngine/lichess_db_standard_rated_2025-05.pgn with 1117 chunks.


Decompressing /content/drive/MyDrive/ChessEngine/lichess_db_standard_rated_2025-06.pgn took 2.26s
Cache not found. Creating chunks for /content/drive/MyDrive/ChessEngine/lichess_db_standard_rated_2025-06.pgn
Chunking took 0.12s
Training /content/drive/MyDrive/ChessEngine/lichess_db_standard_rated_2025-06.pgn with 35 chunks.
{}


ValueError: num_samples should be a positive integer value, but got num_samples=0